<h1>Introduction to the problem sets

<p>
    
    Hi there,

    Thanks for taking an interest in this coding test. There are 7 problem sets:
   
    1) Calculate futures PNL
    2) Run custom summary stats
    3) Plot percentage returns using bokeh
    4) Gather and show economic events
    5) Load a JSON and create a loop to update settings
    6) Create an email function to send attachments (no need to include your password)
    7) IQfeed symbol lookup (no coding)
    
    Please complete all 7 tests using python in this ipython/jupyter notebook. 
    We will compare your results with an internal notebook.

    Please unzip the "mn_test" directory that was sent to you.
    Work in this directory to run the ipython/jupyter notebook titled "mn_test.ipynb".

    Please enter your name info in the code block below.
    
    Thanks and good luck!
</p>

In [1]:
# Example:  PERSON = "John Smith"
PERSON = "John Smith"

In [2]:
PERSON = PERSON.lower().replace(' ', "")
PERSON

'johnsmith'

<h1>1) Calculate futures PNL

If you don't know how to calculate futures pnl, you can use this resource to learn how:
https://www.cmegroup.com/education/courses/introduction-to-futures/calculating-futures-contract-profit-or-loss.html#
    
If you need to know where to find futures contract specs, search the given products "contract spec" webpage on the CME website (we're only using CME products in this example)

For each product in the /data/futures_data.csv file, read it in with pandas and calculate the PNL for each product.

Assume you place a "buy" long trade at the very first price for 5 contracts and hold till the very last price.

(don't worry about rolling futures or weekends, just act as if it's one continuous price data set)

Plot the PNL per product in it's own plot using bokeh.

What is the final PNL per product?

In [3]:
# your code goes here...

<h1>2) Run custom summary stats

We have a summary statistics format that we like to use for daily PNL analysis. 

First, convert the hourly futures data into daily PNLs. 

Then use the following summary_stats() function to produce the analysis.

Run two groups of summary stats: one for ES and another for NQ. 

If your column headers don't match what's in the summary_stats() function, please change them to match.

Use the Ipython display() function to show the final summary_stats dataframe.

In [4]:
import pandas as pd
def summary_stats(df_: pd.DataFrame, column_name="value"):
    
    df_ = df_.dropna(how='all')
    df_["count"] = int(df_["size"].count())
    df_["mean"] = round(df_.pnl.mean())
    df_["min"] = round(df_.pnl.min())
    df_["25(%)"] = round(df_.pnl.quantile(q=0.25))
    df_["50(%)"] = round(df_.pnl.quantile(q=0.5))
    df_["75(%)"] = round(df_.pnl.quantile(q=0.75))
    df_["max"] = round(df_.pnl.max())
    df_["wins"] = df_["pnl"].apply(lambda x: 1 if x > 0 else 0)
    df_["win(%)"] = round(int(len(df_[df_.wins > 0])/df_.shape[0] * 100), 2)
    df_["mean_unit_pnl"] = round(np.sum(df_.pnl)/ np.sum(np.absolute(df_["size"])))
    try:
        df_["std_unit_pnl"] = round(np.std(df_.pnl) / np.sum(np.absolute(df_["size"])))
    except:
        df_["std_unit_pnl"] = np.nan
    df_['average_daily_pnl'] = round(df_.groupby('date').pnl.sum().mean())
    df_['total_dates'] = round(len(df_.date.unique()))
    try:
        df_['std_daily_pnl'] = round(df_.groupby('date').pnl.sum().std())
    except:
        df_['std_daily_pnl'] = np.nan
    try:
        sharpe = np.round((252**0.5)*(df_["average_daily_pnl"] / df_["std_daily_pnl"]), 3)
    except:
        sharpe = np.nan
    df_["sharpe"] = round(sharpe, 2)
    df_["total_pnl"] = round(np.sum(df_.pnl))
    df_["total_volume"] = round(np.sum(np.absolute(df_["size"])))
    df_final = df_.iloc[0][['count', 'mean', 'min', '25(%)',
                            '50(%)', '75(%)', 'max', 'win(%)', 'mean_unit_pnl', 'std_unit_pnl',
                            'average_daily_pnl', 'total_dates', 'std_daily_pnl', 'sharpe',
                            'total_pnl', 'total_volume']].reset_index()
    df_final.columns = ['statistic', column_name]
    df_final = df_final.set_index('statistic')
    return df_final

In [5]:
# your code goes here...

<h1>3) Plot percentage returns using bokeh

You should reuse the code from 1 and 2 to build your datasets and make you plots.

Make a dual line plot of daily % returns per product.

In [6]:
# your code goes here...

<h1>4) Gather and show economic events

Please gather the week's economic events below and sort by earliest date, time.

You will have to use python to scrape a website to do this. 

Use the chromedriver we provided in "driver" folder and use selenium for this task. 

You will have to use a chrome web browser and you may get a chromedriver error. 

Troubleshooting this potential error is a part of the task.

In [7]:
events_url = 'https://www.forexfactory.com/calendar'

In [8]:
# you may find this helpful
import os
EXECUTABLE_PATH = os.path.join(os.getcwd(), "driver", "chromedriver")

<h1>5) Load a JSON and create a loop to update settings

Find the JSON file in /data/RetailSales_CHI.json.

Load this file here and create a function to loop over the "params" and update the following settings:
    
- Change "event_date" to "2021-12-14"
- Change "event_time" to "09:45:00"
- Use event_time variable with a python/pandas timedelta builtin function to add "5 minutes" and save this as the "hedge_start_time"

In [9]:
# your code goes here...

<h1>6) Create an email function to send attachments

You will likely have to consider additional python packages to send an email.

You don't have to include any personal information such as email address and password. 

Just test something that does in fact send an email. 

In [10]:
# you could email an attachment with this naming but not necessary, just show that the email function works
ZIP_DIR_NAME = f"{PERSON}_mn_test"
ZIP_DIR_NAME

'johnsmith_mn_test'

In [11]:
# your code goes here...

<h1>7) IQFeed Symbol Lookup

This task requires no code. Please follow the instructions:

- go to www.iqfeed.net
- go to their symbol lookup search engine
- find the "continuous contract code" for these 5 products:

1. Crude Oil future
2. S&P 500 future
3. Natural Gas Future
4. Eurostoxx 50 future
5. Brent Crude future


In [12]:
iqf_crude_oil = ""
iqf_s_and_p_500 = ""
iqf_nat_gas = ""
iqf_eurostoxx_50 = ""
iqf_brent_crude = ""